<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.0 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [2]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor


import optuna
from optuna.visualization import plot_optimization_history

Evalueringsmetrikker

In [3]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

Importering av relevant data

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd /content/drive/MyDrive/MasterV24/GitHub/
%ls
#from oppdeling_trening_testsett import oppdeling_enzymtype

/content/drive/MyDrive/MasterV24/GitHub
'Bioco_BrixAdjusted: Semiveiledet Regresjonsmetoder.ipynb'
'Bioco_BrixAdjusted: Superveiledet Regresjonsmetoder.ipynb'
'Bioco_Collagen: Klassiske Superveiledet Regresjonsmetoder.ipynb'
'Bioco_Collagen: Semiveiledet Regresjonsmetoder.ipynb'
'Bioco: Databehandling av ekstremverdier.ipynb'
'Bioco: Databehandling av manglende verdier.ipynb'
'Bioco: Dataprosessering, Dataforståelse og Dataundersøkelse.ipynb'
'Bioco_Mw: Semiveiledet Regresjonsmetoder.ipynb'
'Bioco_Mw: Superveiledet Regresjonsmetoder.ipynb'
'Bioco_Smallmolecules: Semiveiledet Regresjonsmetoder.ipynb'
'Bioco_Smallmolecules: Superveiledet Regresjonsmetoder.ipynb'
'Bioco: Variabelundersøkelse.ipynb'
 Oppdeling_trening_testsett.ipynb
 __pycache__/


In [10]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Beskrivelse av datasett

In [6]:
collagen_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatPercent  29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT08           29136 non-null  float64
 11  TT20           29136 non-null  float64
 12  TT12           29136 non-null  float64
 13  Collagen       89 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


Tilfeldighetsfrø

In [33]:
#Tilfeldighetsfrø
random_seed = 123

# Oppdeling av datasett

## Alternativ 1: Fordelt på enzymtyper

In [72]:
 def trening_testsett_oppdeling_enzym(original_df,
                                      filtert_df,
                                      test_andel=0.2,
                                      random_seed=123):




  # Responsvariabel
  responsvariabel = filtert_df.columns[-1]

  # Deler opp markert og umarkert del av datasettet
  data_markert = filtert_df.dropna(subset=[responsvariabel])
  data_umarkert = filtert_df[filtert_df[responsvariabel].isna()]

  # Enzymtyper
  enzym_data = original_df.merge(data_markert,
                                 left_index=True,
                                 right_index=True,
                                 how='inner',
                                 suffixes=('','_drop'))['EnzymeCode']

  # Splitter trening og testdata etter enzymtype
  trening_markert, test_markert, _, _ = train_test_split(data_markert,
                                                         enzym_data,
                                                         test_size=test_andel,
                                                         stratify=enzym_data,
                                                         random_state=random_seed)

  # Definerer trening og testsett for data

  felles_indeks = filtert_df.index.intersection(test_markert.index)
  treningsdata = filtert_df.drop(felles_indeks)
  trening_umarkert = treningsdata[treningsdata[responsvariabel].isna()]

  return trening_markert, test_markert, trening_umarkert, treningsdata

## Alternativ 2: Fordelt på dag og kontinuitet

In [80]:
def trening_testsett_oppdeling_dag(df, andel=0.2):

    # Responsvariabel
    responsvariabel = df.columns[-1]

    # Lager nytt trening og testsett
    data_markert = df.dropna(subset=[responsvariabel])

    trening_markert = copy.deepcopy(data_markert)
    test_markert = pd.DataFrame(columns=list(data_markert.columns))

    # Går gjennom hver dag, hver uke og samler første andel av rader til testsett
    for uke in data_markert.index.isocalendar().week.unique():
        for dag in data_markert.index.isocalendar().day.unique():
            data = data_markert[(data_markert.index.isocalendar().week == uke) &
                      (data_markert.index.isocalendar().day == dag)]
            test_rader = data.shape[0] * andel
            test_data_dag = data.head(round(test_rader))

            test_markert = pd.concat([test_markert, test_data_dag])
            trening_markert.drop(test_data_dag.index, inplace=True)

    felles_indeks = df.index.intersection(test_markert.index)
    treningsdata = df.drop(felles_indeks)
    trening_umarkert = treningsdata[treningsdata[responsvariabel].isna()]

    return trening_markert, test_markert, trening_umarkert, treningsdata

In [226]:
def trening_testsett_oppdeling_dag_2(original_df,
                                     filtert_df,
                                     test_andel=0.2):

    # Responsvariabel
    responsvariabel = filtert_df.columns[-1]

    # Lager nytt trening og testsett
    data_markert = filtert_df.dropna(subset=[responsvariabel])

    trening_markert = copy.deepcopy(data_markert)
    test_markert = pd.DataFrame(columns=list(data_markert.columns))

    # Råmateriale type
    råmateriale_data = original_df.merge(data_markert,
                                         left_index=True,
                                         right_index=True,
                                         how='inner',
                                         suffixes=('','_drop'))['RawMaterialMix']
    data_markert['RawMaterialMix'] = råmateriale_data


    # Går gjennom hver dag, hver uke og samler første andel av rader til testsett
    for uke in data_markert.index.isocalendar().week.unique():
        for dag in data_markert.index.isocalendar().day.unique():
            data = data_markert[(data_markert.index.isocalendar().week == uke) &
                      (data_markert.index.isocalendar().day == dag)]
            test_rader = round((data.shape[0] * test_andel))
            test_data_dag = data.head(test_rader)

            # Sjekker første instans av "Turkey" eller "Chicken" i foreløpig testsett
            krav = (test_data_dag["RawMaterialMix"] == "Turkey") | (test_data_dag["RawMaterialMix"] == "Chicken")

            # Hvis testsettet inneholder "Turkey" eller "Chicken", legges det
            # til en rad, da første rad med gjelende utelukkes
            if krav.any():
              test_rader += 1

              # Finner mengde testsett uten "Turkey" eller "Chicken"
              while krav.any():
                test_data_dag = data.loc[krav.idxmax():].head(test_rader).copy()
                test_data_dag = test_data_dag.drop(index = krav.idxmax())

                # Finner nest beste mengde testsett uten "Turkey" eller "Chicken"
                if (test_data_dag.shape[0] < test_rader) & (test_data_dag.index[-1] == data.index[-1]):
                  test_rader -= 1
                  test_data_dag = data.head(test_rader).copy()

                krav = (test_data_dag["RawMaterialMix"] == "Turkey") | (test_data_dag["RawMaterialMix"] == "Chicken")

            # Dropper "RawMaterialMix" som kolonne
            test_data_dag = test_data_dag.drop(columns = ['RawMaterialMix'])

            # Legger til i fullstendig testsett og fjerner fra treningssett
            test_markert = pd.concat([test_markert, test_data_dag])
            trening_markert = trening_markert.drop(test_data_dag.index.copy())

    felles_indeks = filtert_df.index.intersection(test_markert.index)
    treningsdata = filtert_df.drop(felles_indeks.copy())
    trening_umarkert = treningsdata[treningsdata[responsvariabel].isna()]

    return trening_markert, test_markert, trening_umarkert, treningsdata

# NIR målinger inkludert

## Oppdeling av datasett for trening og testing

### Alternativ 1: Fordelt på Enzymtype

In [76]:
test_andel = 0.2
collagen_trening, collagen_test, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                                                         collagen_data,
                                                                         test_andel=test_andel)

X_trening = collagen_trening.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening = collagen_trening.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

**RandomForestRegressor**

Hyperparamter optimalisering med Optuna

In [34]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rfr_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  rmse_test = rmse(y_test, y_test_prediksjon)
  return rmse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-14 09:36:10,513] A new study created in memory with name: no-name-11a72819-b9ee-429d-af97-5080d4581802
[I 2024-02-14 09:36:10,764] Trial 0 finished with value: 4.115387894790196 and parameters: {'n_estimators': 109, 'max_depth': 4}. Best is trial 0 with value: 4.115387894790196.
[I 2024-02-14 09:36:11,046] Trial 1 finished with value: 3.9737645204599037 and parameters: {'n_estimators': 113, 'max_depth': 9}. Best is trial 1 with value: 3.9737645204599037.
[I 2024-02-14 09:36:11,468] Trial 2 finished with value: 3.9517498727134304 and parameters: {'n_estimators': 170, 'max_depth': 9}. Best is trial 2 with value: 3.9517498727134304.
[I 2024-02-14 09:36:11,868] Trial 3 finished with value: 3.9898576735940448 and parameters: {'n_estimators': 177, 'max_depth': 8}. Best is trial 2 with value: 3.9517498727134304.
[I 2024-02-14 09:36:12,145] Trial 4 finished with value: 4.087127236681098 and parameters: {'n_estimators': 122, 'max_depth': 4}. Best is trial 2 with value: 3.951749872713

Evauleringer av beste modell

In [35]:
def detailed_objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rfr_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rfr_pipeline.predict(X_test)

  y_trening_prediksjon = rfr_pipeline.predict(X_trening)

  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)


  rmse_trening = rmse(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

rmse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mae_test_resultat = detailed_objective(study.best_trial)[2]
mape_test_resultat = detailed_objective(study.best_trial)[3]

rmse_trening_resultat = detailed_objective(study.best_trial)[4]
r2_trening_resultat = detailed_objective(study.best_trial)[5]
mae_trening_resultat = detailed_objective(study.best_trial)[6]
mape_trening_resultat = detailed_objective(study.best_trial)[7]

collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

beste_parametere = study.best_params

resultater = [test_andel,
              rmse_test_resultat,
              r2_test_resultat,
              mae_test_resultat,
              mape_test_resultat,
              rmse_trening_resultat,
              r2_trening_resultat,
              mae_trening_resultat,
              mape_trening_resultat,
              beste_parametere]

collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater
print(collagen_rfr_resultater_enzym)

   Test andel  RMSE test  R2 test  MAE test  MAPE test  RMSE trening  \
0         0.2   3.920317  0.55775  3.398218   0.172559      1.228225   

   R2 trening  MAE trening  MAPE trening  \
0    0.940468     0.941896      0.046241   

                        Beste parametere  
0  {'n_estimators': 151, 'max_depth': 9}  


### Alternativ 1: Fordelt på enzymtyper og forskjellige testandeler

In [ ]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

test_andeler = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for test_andel in test_andeler:
  collagen_trening, collagen_test, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                                                           collagen_data,
                                                                           test_andel=test_andel)

  X_trening = collagen_trening.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]
  y_trening = collagen_trening.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  def objective(trial):
    parametere = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 1, 20)
    }
    rf_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
    ])

    rf_pipeline.fit(X_trening, y_trening)
    y_test_prediksjon = rf_pipeline.predict(X_test)
    rmse_test = rmse(y_test, y_test_prediksjon)
    return rmse_test

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=150)


  def detailed_objective(trial):
    parametere = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 1, 20)
    }
    rfr_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
    ])

    rfr_pipeline.fit(X_trening, y_trening)
    y_test_prediksjon = rfr_pipeline.predict(X_test)
    y_trening_prediksjon = rfr_pipeline.predict(X_trening)

    rmse_test = rmse(y_test, y_test_prediksjon)
    r2_test = r2_score(y_test, y_test_prediksjon)
    mae_test = mean_absolute_error(y_test, y_test_prediksjon)
    mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)


    rmse_trening = rmse(y_trening, y_trening_prediksjon)
    r2_trening = r2_score(y_trening, y_trening_prediksjon)
    mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
    mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

    return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

  rmse_test_resultat = detailed_objective(study.best_trial)[0]
  r2_test_resultat = detailed_objective(study.best_trial)[1]
  mae_test_resultat = detailed_objective(study.best_trial)[2]
  mape_test_resultat = detailed_objective(study.best_trial)[3]

  rmse_trening_resultat = detailed_objective(study.best_trial)[4]
  r2_trening_resultat = detailed_objective(study.best_trial)[5]
  mae_trening_resultat = detailed_objective(study.best_trial)[6]
  mape_trening_resultat = detailed_objective(study.best_trial)[7]

  beste_parametere = study.best_params

  resultater = [test_andel,
                rmse_test_resultat,
                r2_test_resultat,
                mae_test_resultat,
                mape_test_resultat,
                rmse_trening_resultat,
                r2_trening_resultat,
                mae_trening_resultat,
                mape_trening_resultat,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

In [40]:
collagen_rfr_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.10,4.190353,0.542093,3.494924,0.160842,1.166512,0.948053,0.938079,0.045484,"{'n_estimators': 272, 'max_depth': 9}"
1,0.15,3.828881,0.431979,3.345650,0.150339,1.209629,0.947217,0.946869,0.047198,"{'n_estimators': 123, 'max_depth': 13}"
2,0.20,3.895571,0.563316,3.363347,0.170870,1.202712,0.942915,0.928531,0.045396,"{'n_estimators': 295, 'max_depth': 9}"
3,0.25,3.618656,0.552904,3.165244,0.155349,1.245590,0.941987,0.981127,0.047641,"{'n_estimators': 252, 'max_depth': 9}"
4,0.30,3.821844,0.560233,3.303940,0.165966,1.220310,0.940204,0.979738,0.046817,"{'n_estimators': 128, 'max_depth': 12}"
5,0.35,3.979178,0.473580,3.434009,0.167673,1.168245,0.947071,0.939472,0.045831,"{'n_estimators': 281, 'max_depth': 11}"
6,0.40,4.044737,0.408014,3.521440,0.168727,1.234467,0.943674,0.953312,0.047409,"{'n_estimators': 300, 'max_depth': 15}"
7,0.45,3.833715,0.466478,3.309692,0.156246,1.270307,0.939177,0.965758,0.049142,"{'n_estimators': 276, 'max_depth': 10}"
8,0.50,3.643855,0.513075,3.064735,0.144990,1.294239,0.938980,1.053585,0.053105,"{'n_estimators': 242, 'max_depth': 12}"


### Alternativ 2: Fordelt på dag og kontinuitet

In [233]:
test_andel = 0.2
collagen_trening, collagen_test, _, _ = trening_testsett_oppdeling_dag_2(rå_data,
                                                                         collagen_data,
                                                                         test_andel=test_andel)

X_trening = collagen_trening.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening = collagen_trening.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

<ipython-input-226-0fb3ac590883>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**RandomForestRegressor**

Hyperparameteroptimalisering med Optuna

In [235]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rfr_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  rmse_test = rmse(y_test, y_test_prediksjon)
  return rmse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-14 15:05:47,752] A new study created in memory with name: no-name-6f131ca0-952f-4621-be49-f2aebe17dd27
[I 2024-02-14 15:05:48,177] Trial 0 finished with value: 2.830409383199701 and parameters: {'n_estimators': 166, 'max_depth': 10}. Best is trial 0 with value: 2.830409383199701.
[I 2024-02-14 15:05:48,512] Trial 1 finished with value: 2.791428893732551 and parameters: {'n_estimators': 131, 'max_depth': 9}. Best is trial 1 with value: 2.791428893732551.
[I 2024-02-14 15:05:48,998] Trial 2 finished with value: 2.903895905963613 and parameters: {'n_estimators': 146, 'max_depth': 4}. Best is trial 1 with value: 2.791428893732551.
[I 2024-02-14 15:05:49,498] Trial 3 finished with value: 2.7749234415056647 and parameters: {'n_estimators': 103, 'max_depth': 8}. Best is trial 3 with value: 2.7749234415056647.
[I 2024-02-14 15:05:49,981] Trial 4 finished with value: 2.811408961607794 and parameters: {'n_estimators': 124, 'max_depth': 7}. Best is trial 3 with value: 2.774923441505664

In [236]:
def detailed_objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
  ])

  rfr_pipeline.fit(X_trening, y_trening)
  y_test_prediksjon = rfr_pipeline.predict(X_test)

  y_trening_prediksjon = rfr_pipeline.predict(X_trening)

  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)


  rmse_trening = rmse(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

rmse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mae_test_resultat = detailed_objective(study.best_trial)[2]
mape_test_resultat = detailed_objective(study.best_trial)[3]

rmse_trening_resultat = detailed_objective(study.best_trial)[4]
r2_trening_resultat = detailed_objective(study.best_trial)[5]
mae_trening_resultat = detailed_objective(study.best_trial)[6]
mape_trening_resultat = detailed_objective(study.best_trial)[7]

collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

beste_parametere = study.best_params

resultater = [test_andel,
              rmse_test_resultat,
              r2_test_resultat,
              mae_test_resultat,
              mape_test_resultat,
              rmse_trening_resultat,
              r2_trening_resultat,
              mae_trening_resultat,
              mape_trening_resultat,
              beste_parametere]

collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater
print(collagen_rfr_resultater_enzym)

   Test andel  RMSE test   R2 test  MAE test  MAPE test  RMSE trening  \
0         0.2   2.756939  0.678066  2.543116   0.119563      1.297451   

   R2 trening  MAE trening  MAPE trening  \
0    0.939915     1.053718      0.051226   

                         Beste parametere  
0  {'n_estimators': 107, 'max_depth': 10}  


### Alternativ 2: Fordelt på dag og forkjellige testandeler

In [237]:
collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Test andel",
                                                      "RMSE test",
                                                      "R2 test",
                                                      "MAE test",
                                                      "MAPE test",
                                                      "RMSE trening",
                                                      "R2 trening",
                                                      "MAE trening",
                                                      "MAPE trening",
                                                      "Beste parametere"])

test_andeler = [0.1, 0.2, 0.3, 0.4, 0.5]

for test_andel in test_andeler:
  collagen_trening, collagen_test, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                                                           collagen_data,
                                                                           test_andel=test_andel)

  X_trening = collagen_trening.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]
  y_trening = collagen_trening.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  def objective(trial):
    parametere = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 1, 20)
    }
    rf_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
    ])

    rf_pipeline.fit(X_trening, y_trening)
    y_test_prediksjon = rf_pipeline.predict(X_test)
    rmse_test = rmse(y_test, y_test_prediksjon)
    return rmse_test

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  def detailed_objective(trial):
    parametere = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 1, 20)
    }
    rfr_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
    ])

    rfr_pipeline.fit(X_trening, y_trening)
    y_test_prediksjon = rfr_pipeline.predict(X_test)
    y_trening_prediksjon = rfr_pipeline.predict(X_trening)

    rmse_test = rmse(y_test, y_test_prediksjon)
    r2_test = r2_score(y_test, y_test_prediksjon)
    mae_test = mean_absolute_error(y_test, y_test_prediksjon)
    mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)


    rmse_trening = rmse(y_trening, y_trening_prediksjon)
    r2_trening = r2_score(y_trening, y_trening_prediksjon)
    mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
    mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

    return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

  rmse_test_resultat = detailed_objective(study.best_trial)[0]
  r2_test_resultat = detailed_objective(study.best_trial)[1]
  mae_test_resultat = detailed_objective(study.best_trial)[2]
  mape_test_resultat = detailed_objective(study.best_trial)[3]

  rmse_trening_resultat = detailed_objective(study.best_trial)[4]
  r2_trening_resultat = detailed_objective(study.best_trial)[5]
  mae_trening_resultat = detailed_objective(study.best_trial)[6]
  mape_trening_resultat = detailed_objective(study.best_trial)[7]

  beste_parametere = study.best_params

  resultater = [test_andel,
                rmse_test_resultat,
                r2_test_resultat,
                mae_test_resultat,
                mape_test_resultat,
                rmse_trening_resultat,
                r2_trening_resultat,
                mae_trening_resultat,
                mape_trening_resultat,
                beste_parametere]

  collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

[I 2024-02-14 15:12:45,630] A new study created in memory with name: no-name-b87d57b4-5739-486d-9984-d15c4a4cc54d
[I 2024-02-14 15:12:45,956] Trial 0 finished with value: 4.386530295858305 and parameters: {'n_estimators': 148, 'max_depth': 4}. Best is trial 0 with value: 4.386530295858305.
[I 2024-02-14 15:12:46,635] Trial 1 finished with value: 4.193124646115002 and parameters: {'n_estimators': 271, 'max_depth': 9}. Best is trial 1 with value: 4.193124646115002.
[I 2024-02-14 15:12:47,243] Trial 2 finished with value: 4.632807866724967 and parameters: {'n_estimators': 283, 'max_depth': 2}. Best is trial 1 with value: 4.193124646115002.
[I 2024-02-14 15:12:47,618] Trial 3 finished with value: 4.638477774085174 and parameters: {'n_estimators': 254, 'max_depth': 2}. Best is trial 1 with value: 4.193124646115002.
[I 2024-02-14 15:12:48,517] Trial 4 finished with value: 4.1985389137204825 and parameters: {'n_estimators': 263, 'max_depth': 10}. Best is trial 1 with value: 4.193124646115002.

In [239]:
collagen_rfr_resultater_dag

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,4.190353,0.542093,3.494924,0.160842,1.166512,0.948053,0.938079,0.045484,"{'n_estimators': 272, 'max_depth': 9}"
1,0.2,3.895613,0.563306,3.368638,0.171006,1.202361,0.942948,0.930603,0.045403,"{'n_estimators': 278, 'max_depth': 9}"
2,0.3,3.827516,0.558927,3.299989,0.166255,1.219238,0.940309,0.981398,0.046810,"{'n_estimators': 105, 'max_depth': 12}"
3,0.4,4.044737,0.408014,3.521440,0.168727,1.234467,0.943674,0.953312,0.047409,"{'n_estimators': 300, 'max_depth': 19}"
4,0.5,3.643879,0.513068,3.064684,0.144988,1.294150,0.938989,1.053166,0.053090,"{'n_estimators': 242, 'max_depth': 16}"


# NIR målinger ekskludert